In [1]:
!pip -q install transformers datasets accelerate faiss-cpu gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 79.7 MB/s eta 0:00:00


In [2]:
import torch, numpy as np
from transformers import EsmTokenizer, EsmForMaskedLM

device = "cuda" if torch.cuda.is_available() else "cpu"

PROT_MODEL = "westlake-repl/SaProt_650M_AF2"
prot_tok = EsmTokenizer.from_pretrained(PROT_MODEL)
prot_mlm = EsmForMaskedLM.from_pretrained(PROT_MODEL, torch_dtype=torch.float16).to(device)
prot_mlm.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of EsmForMaskedLM were not initialized from the model checkpoint at westlake-repl/SaProt_650M_AF2 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmForMaskedLM(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(446, 1280, padding_idx=1)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-32): 33 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=1280, out_features=1280, bias=True)
              (key): Linear(in_features=1280, out_features=1280, bias=True)
              (value): Linear(in_features=1280, out_features=1280, bias=True)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=1280, out_features=1280, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (LayerNorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          )
          (intermediate): EsmIntermediate(
            (dense): Linear(in_features=

In [3]:
def aa_to_saprot_seq(aa: str) -> str:
    aa = aa.strip().upper().replace(" ", "")

    return "".join([c + "#" for c in aa if c.isalpha()])

@torch.no_grad()
def embed_protein(aa_seq: str) -> np.ndarray:
    sap = aa_to_saprot_seq(aa_seq)
    inputs = prot_tok(sap, return_tensors="pt", truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    out = prot_mlm(**inputs, output_hidden_states=True, return_dict=True)
    last = out.hidden_states[-1]                          # [B, T, H]
    mask = inputs["attention_mask"].unsqueeze(-1)         # [B, T, 1]
    pooled = (last * mask).sum(1) / mask.sum(1)           # mean pooling
    vec = pooled[0].float().cpu().numpy()
    vec = vec / (np.linalg.norm(vec) + 1e-12)             # normalize
    return vec


In [4]:
from transformers import AutoTokenizer, AutoModel

MOL_MODEL = "DeepChem/ChemBERTa-77M-MLM"
mol_tok = AutoTokenizer.from_pretrained(MOL_MODEL)
mol_enc = AutoModel.from_pretrained(MOL_MODEL, torch_dtype=torch.float16).to(device)
mol_enc.eval()

@torch.no_grad()
def embed_smiles(smiles: str) -> np.ndarray:
    inputs = mol_tok(smiles, return_tensors="pt", truncation=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    out = mol_enc(**inputs, return_dict=True)
    last = out.last_hidden_state
    mask = inputs["attention_mask"].unsqueeze(-1)
    pooled = (last * mask).sum(1) / mask.sum(1)
    vec = pooled[0].float().cpu().numpy()
    vec = vec / (np.linalg.norm(vec) + 1e-12)
    return vec


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
!pip -q uninstall -y huggingface-hub
!pip -q install -U "huggingface-hub>=0.34.0,<1.0" "datasets>=2.18.0,<3"



model.safetensors:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 327.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 441.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 367.7 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [8]:
!hf auth login




    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `ai` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing

In [9]:
from datasets import load_dataset

ds_stream = load_dataset("eve-bio/drug-target-activity", split="train", streaming=True, token=True)
first = next(iter(ds_stream))
print(first)
print(first.keys())


{'assay_id': 'P00316', 'target_id': 'ROS1', 'compound_id': 'EB000590', 'mode': 'Binding', 'mechanism': 'Competition Binding', 'outcome_is_active': True, 'outcome_potency_pxc50': 11.8, 'outcome_max_activity': 99.3, 'observed_max': 100.0, 'is_quantified': True, 'frequency_flag': False, 'viability_flag': False, 'pxc50_modifier': '>', 'slope': 0.6, 'asymp_min': 58.0, 'asymp_max': 99.3, 'assay__technology': 'TR-FRET', 'target__class': 'Kinase', 'target__gene': 'ROS1', 'target__uniprot_id': 'P08922', 'target__is_mutant': False, 'target__wildtype_id': 'ROS1', 'target__name': 'Proto-oncogene tyrosine-protein kinase ROS', 'compound__name': 'Lorlatinib', 'compound__smiles': 'C[C@H]1OC2=C(N)N=CC(=C2)C2=C(C#N)N(C)N=C2CN(C)C(=O)C2=C1C=C(F)C=C2', 'compound__drugbank_id': 'DB12130', 'compound__cas': '1454846-35-5', 'compound__unii': 'OSP71S83EU', 'compound__inchikey': 'IIXWYSCJSQVBQM-LLVKDONJSA-N', 'progressed': True, 'release': '8'}
dict_keys(['assay_id', 'target_id', 'compound_id', 'mode', 'mechani

In [10]:
import pandas as pd
import time, requests
from tqdm import tqdm

rows = []
for i, r in enumerate(ds_stream):
  rows.append(r)
  if i >= 5000: break

df = pd.DataFrame(rows)

df = df[["target__uniprot_id", "compound__smiles", "outcome_is_active", "outcome_potency_pxc50"]].dropna()

df_pos = df[df["outcome_is_active"] == True].copy()

session = requests.Session()

def fetch_uniprot_seq(acc: str) -> str | None:
    url = f"https://rest.uniprot.org/uniprotkb/{acc}.fasta"
    r = session.get(url, timeout=30)
    if r.status_code != 200:
        return None
    seq = "".join([ln.strip() for ln in r.text.splitlines() if not ln.startswith(">")])
    return seq if seq else None

uniq_acc = sorted(df_pos["target__uniprot_id"].unique().tolist())

seq_map = {}
for acc in tqdm(uniq_acc, desc="Downloading UniProt FASTA"):
    seq_map[acc] = fetch_uniprot_seq(acc)
    time.sleep(0.05)

df_pos["protein_seq"] = df_pos["target__uniprot_id"].map(seq_map)
df_pos = df_pos.dropna(subset=["protein_seq"])

print("pairs:", len(df_pos), "unique proteins:", df_pos["target__uniprot_id"].nunique(), "unique smiles:", df_pos["compound__smiles"].nunique())
df_pos.head()


pairs: 4915 unique proteins: 171 unique smiles: 558


,target__uniprot_id,compound__smiles,outcome_is_active,outcome_potency_pxc50,protein_seq
0,P08922,C[C@H]1OC2=C(N)N=CC(=C2)C2=C(C#N)N(C)N=C2CN(C)...,True,11.8,MKNIYCLIPKLVNFATLGCLWISVVQCTVLNSCLKSCVTNLGQQLD...
1,P14416,CS(O)(=O)=O.[H][C@@]12CCCN1C(=O)[C@H](CC1=CC=C...,True,11.6,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...
2,P35348,[H][C@@]12C[C@@H](C)[C@]34C=C(C)[C@H](OC(=O)C(...,True,11.1,MVFLSGNASDSSNCTQPPAPVNISKAILLGVILGGLILFGVLGNIL...
3,P07947,CC1=NC(NC2=NC=C(S2)C(=O)NC2=C(C)C=CC=C2Cl)=CC(...,True,11.1,MGCIKSKENKSPAIKYRPENTPEPVSTSVSHYGAEPTTVSPCPSSS...
4,Q15722,[H][C@@]12C[C@@H](C)[C@]34C=C(C)[C@H](OC(=O)C(...,True,10.8,MNTTSSAAPPSLGVEFISLLAIILLSVALAVGLPGNSFVVWSILKR...


In [11]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_pos)

https://docs.google.com/spreadsheets/d/15DcypYnrA90FbQtWGMkNexqY7m81PxEpTTz_enX7ROM/edit#gid=0


In [12]:
import torch
print("cuda available:", torch.cuda.is_available())
print("device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


cuda available: False
device: cpu


In [13]:
!pip -q install transformers faiss-cpu

import numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
import faiss
from tqdm import tqdm
from transformers import EsmTokenizer, EsmForMaskedLM, AutoTokenizer, AutoModel

# ---- 0) checks ----
need_cols = ["target__uniprot_id", "protein_seq", "compound__smiles"]
for c in need_cols:
    if c not in df_pos.columns:
        raise RuntimeError(f"df_pos 缺少列 {c}，你现在有: {df_pos.columns.tolist()}")

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

if device == "cpu":
    MAX_PROTS, MAX_MOLS, MAX_PAIRS, STEPS, BATCH = 5, 50, 300, 80, 32
else:
    MAX_PROTS, MAX_MOLS, MAX_PAIRS, STEPS, BATCH = 200, 2000, 20000, 800, 256

print("MAX_PROTS, MAX_MOLS, MAX_PAIRS, STEPS, BATCH =", MAX_PROTS, MAX_MOLS, MAX_PAIRS, STEPS, BATCH)

# ---- 1) build candidate sets ----
prot_df = df_pos[["target__uniprot_id","protein_seq"]].drop_duplicates("target__uniprot_id").head(MAX_PROTS)
mol_df  = df_pos[["compound__smiles"]].drop_duplicates("compound__smiles").head(MAX_MOLS)

prot_ids = prot_df["target__uniprot_id"].tolist()
prot_seq = dict(zip(prot_df["target__uniprot_id"], prot_df["protein_seq"]))
smiles_list = mol_df["compound__smiles"].tolist()

prot2i = {p:i for i,p in enumerate(prot_ids)}
mol2i  = {s:i for i,s in enumerate(smiles_list)}

df_train = df_pos[df_pos["target__uniprot_id"].isin(prot_ids) & df_pos["compound__smiles"].isin(smiles_list)]
df_train = df_train.sample(n=min(MAX_PAIRS, len(df_train)), random_state=0).reset_index(drop=True)

pair_p = np.array([prot2i[p] for p in df_train["target__uniprot_id"]], dtype=np.int64)
pair_m = np.array([mol2i[s]  for s in df_train["compound__smiles"]], dtype=np.int64)

print("unique proteins:", len(prot_ids), "unique mols:", len(smiles_list), "train pairs:", len(df_train))

# ---- 2) load encoders ----
PROT_MODEL = "westlake-repl/SaProt_650M_AF2"
MOL_MODEL  = "DeepChem/ChemBERTa-77M-MLM"

prot_tok = EsmTokenizer.from_pretrained(PROT_MODEL)
prot_mlm = EsmForMaskedLM.from_pretrained(PROT_MODEL).to(device).eval()

mol_tok = AutoTokenizer.from_pretrained(MOL_MODEL)
mol_enc = AutoModel.from_pretrained(MOL_MODEL).to(device).eval()

def aa_to_saprot_seq(aa: str) -> str:
    aa = aa.strip().upper().replace(" ", "")
    return "".join([c + "#" for c in aa if c.isalpha()])  # AA-only

@torch.no_grad()
def protein_base_vec(aa_seq: str) -> np.ndarray:
    sap = aa_to_saprot_seq(aa_seq)
    x = prot_tok(sap, return_tensors="pt", truncation=True, max_length=1024)
    x = {k:v.to(device) for k,v in x.items()}
    out = prot_mlm(**x, output_hidden_states=True, return_dict=True)
    last = out.hidden_states[-1]                 # [1,T,H]
    mask = x["attention_mask"].unsqueeze(-1)     # [1,T,1]
    pooled = (last * mask).sum(1) / mask.sum(1)  # mean pooling
    return pooled[0].detach().float().cpu().numpy()

@torch.no_grad()
def mol_base_vec(smiles: str) -> np.ndarray:
    x = mol_tok(smiles, return_tensors="pt", truncation=True, max_length=256)
    x = {k:v.to(device) for k,v in x.items()}
    out = mol_enc(**x, return_dict=True)
    last = out.last_hidden_state
    mask = x["attention_mask"].unsqueeze(-1)
    pooled = (last * mask).sum(1) / mask.sum(1)
    return pooled[0].detach().float().cpu().numpy()

# ---- 3) precompute base embeddings ----
print("Embedding proteins...")
prot_base = np.stack([protein_base_vec(prot_seq[p]) for p in tqdm(prot_ids)]).astype("float32")
print("Embedding molecules...")
mol_base  = np.stack([mol_base_vec(s) for s in tqdm(smiles_list)]).astype("float32")

Hp, Hm = prot_base.shape[1], mol_base.shape[1]
print("Hp, Hm =", Hp, Hm)

prot_base_t = torch.from_numpy(prot_base).to(device)
mol_base_t  = torch.from_numpy(mol_base).to(device)

# ---- 4) train tiny projections (this aligns spaces) ----
d = 256
proj_p = nn.Linear(Hp, d, bias=False).to(device)
proj_m = nn.Linear(Hm, d, bias=False).to(device)
opt = torch.optim.AdamW(list(proj_p.parameters()) + list(proj_m.parameters()), lr=1e-3, weight_decay=1e-4)
temp = 0.07

proj_p.train(); proj_m.train()
for step in range(STEPS):
    idx = np.random.randint(0, len(pair_p), size=min(BATCH, len(pair_p)))
    bp = torch.from_numpy(pair_p[idx]).to(device)
    bm = torch.from_numpy(pair_m[idx]).to(device)

    p = F.normalize(proj_p(prot_base_t[bp]), dim=-1)
    m = F.normalize(proj_m(mol_base_t[bm]),  dim=-1)

    logits = (p @ m.T) / temp
    labels = torch.arange(logits.size(0), device=device)
    loss = 0.5*(F.cross_entropy(logits, labels) + F.cross_entropy(logits.T, labels))

    opt.zero_grad(); loss.backward(); opt.step()

    if (step+1) % max(1, STEPS//5) == 0:
        print(f"step {step+1}/{STEPS} loss={loss.item():.4f}")

# ---- 5) build FAISS indices in shared space ----
proj_p.eval(); proj_m.eval()
with torch.no_grad():
    P_emb = F.normalize(proj_p(prot_base_t), dim=-1).float().cpu().numpy().astype("float32")
    M_emb = F.normalize(proj_m(mol_base_t),  dim=-1).float().cpu().numpy().astype("float32")

mol_index  = faiss.IndexFlatIP(d); mol_index.add(M_emb)
prot_index = faiss.IndexFlatIP(d); prot_index.add(P_emb)

def search_mols_by_protein(uniprot_id, topk=10):
    i = prot2i[uniprot_id]
    D, I = mol_index.search(P_emb[i:i+1], int(topk))
    return [(float(D[0][j]), smiles_list[int(I[0][j])]) for j in range(int(topk))]

def search_prots_by_smiles(smiles, topk=10):
    i = mol2i[smiles]
    D, I = prot_index.search(M_emb[i:i+1], int(topk))
    return [(float(D[0][j]), prot_ids[int(I[0][j])]) for j in range(int(topk))]

print("✅ Done. Now you have: prot_ids, smiles_list, prot_seq, mol_index, prot_index, search_*")


device: cpu
MAX_PROTS, MAX_MOLS, MAX_PAIRS, STEPS, BATCH = 5 50 300 80 32
unique proteins: 5 unique mols: 50 train pairs: 53


Some weights of EsmForMaskedLM were not initialized from the model checkpoint at westlake-repl/SaProt_650M_AF2 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding proteins...


100%|██████████| 5/5 [02:21<00:00, 28.26s/it]


Embedding molecules...


100%|██████████| 50/50 [00:01<00:00, 39.22it/s]


Hp, Hm = 1280 384
step 16/80 loss=3.0619
step 32/80 loss=2.8060
step 48/80 loss=2.7209
step 64/80 loss=3.0289
step 80/80 loss=2.8270
✅ Done. Now you have: prot_ids, smiles_list, prot_seq, mol_index, prot_index, search_*


In [14]:
print("Protein IDs:", prot_ids)
print("Example protein:", prot_ids[0])
print("Top-5 mols:", search_mols_by_protein(prot_ids[0], topk=5))

print("\nExample SMILES:", smiles_list[0])
print("Top-5 prots:", search_prots_by_smiles(smiles_list[0], topk=5))


Protein IDs: ['P08922', 'P14416', 'P35348', 'P07947', 'Q15722']
Example protein: P08922
Top-5 mols: [(0.593325674533844, 'CCN1CCN(CC1)C1=CC=C(NC2=CC(=NC=N2)N(C)C(=O)NC2=C(Cl)C(OC)=CC(OC)=C2Cl)C=C1'), (0.5858355760574341, 'C[C@H]1OC2=C(N)N=CC(=C2)C2=C(C#N)N(C)N=C2CN(C)C(=O)C2=C1C=C(F)C=C2'), (0.583391547203064, 'COC1=C(NC2=NC=CC(=N2)C2=CN(C)C3=C2C=CC=C3)C=C(NC(=O)C=C)C(=C1)N(C)CCN(C)C'), (0.5801397562026978, 'N[C@@H](CC1=CC(I)=C(OC2=CC(I)=C(O)C=C2)C(I)=C1)C(O)=O'), (0.5737218260765076, 'COC1=CC2=C(C=C1OC)C(OC1=CC=C(NC(=O)C3(CC3)C(=O)NC3=CC=C(F)C=C3)C=C1)=CC=N2')]

Example SMILES: C[C@H]1OC2=C(N)N=CC(=C2)C2=C(C#N)N(C)N=C2CN(C)C(=O)C2=C1C=C(F)C=C2
Top-5 prots: [(0.5858355760574341, 'P08922'), (0.3503883481025696, 'P07947'), (0.09959251433610916, 'Q15722'), (0.05003920942544937, 'P14416'), (0.017553189769387245, 'P35348')]


In [15]:
from tqdm import tqdm

df_eval = df_pos[df_pos["target__uniprot_id"].isin(prot_ids) & df_pos["compound__smiles"].isin(smiles_list)].copy()

pos_mols = df_eval.groupby("target__uniprot_id")["compound__smiles"].apply(set).to_dict()
pos_prots = df_eval.groupby("compound__smiles")["target__uniprot_id"].apply(set).to_dict()

def recall_prot2mol(k=10):
    accs = list(pos_mols.keys())
    hit = 0
    for acc in tqdm(accs, desc=f"P->M Recall@{k}"):
        rec = [s for _, s in search_mols_by_protein(acc, topk=k)]
        if len(pos_mols[acc].intersection(rec)) > 0:
            hit += 1
    return hit / max(1, len(accs))

def recall_mol2prot(k=10):
    smis = list(pos_prots.keys())
    hit = 0
    for smi in tqdm(smis, desc=f"M->P Recall@{k}"):
        rec = [p for _, p in search_prots_by_smiles(smi, topk=k)]
        if len(pos_prots[smi].intersection(rec)) > 0:
            hit += 1
    return hit / max(1, len(smis))

print("P->M Recall@1 :", recall_prot2mol(1))
print("P->M Recall@5 :", recall_prot2mol(5))
print("P->M Recall@10:", recall_prot2mol(10))
print("M->P Recall@1 :", recall_mol2prot(1))
print("M->P Recall@5 :", recall_mol2prot(5))
print("M->P Recall@10:", recall_mol2prot(10))


P->M Recall@1: 100%|██████████| 5/5 [00:00<00:00, 8325.34it/s]


P->M Recall@1 : 0.8


P->M Recall@5: 100%|██████████| 5/5 [00:00<00:00, 2641.91it/s]


P->M Recall@5 : 1.0


P->M Recall@10: 100%|██████████| 5/5 [00:00<00:00, 10082.46it/s]


...

M->P Recall@1 : 1.0


M->P Recall@5: 100%|██████████| 29/29 [00:00<00:00, 17411.22it/s]


M->P Recall@5 : 1.0


M->P Recall@10: 100%|██████████| 29/29 [00:00<00:00, 13306.51it/s]

M->P Recall@10: 1.0


In [16]:
import numpy as np
import pandas as pd

# 只保留pair关系（去重）
pairs = df_pos[["target__uniprot_id","compound__smiles","protein_seq"]].dropna().drop_duplicates()

# 按蛋白分组 split
all_prots = pairs["target__uniprot_id"].unique()
rng = np.random.RandomState(0)
rng.shuffle(all_prots)
cut = int(0.8 * len(all_prots))
train_prots = set(all_prots[:cut])
test_prots  = set(all_prots[cut:])

train_pairs = pairs[pairs["target__uniprot_id"].isin(train_prots)].copy()
test_pairs  = pairs[pairs["target__uniprot_id"].isin(test_prots)].copy()

print("train pairs:", len(train_pairs), "test pairs:", len(test_pairs),
      "train prots:", len(train_prots), "test prots:", len(test_prots))


train pairs: 3686 test pairs: 909 train prots: 136 test prots: 35


In [17]:
for name in ["df_pos", "pairs", "train_pairs", "test_pairs"]:
    print(name, "->", "✅ exists" if name in globals() else "❌ missing")


df_pos -> ✅ exists
pairs -> ✅ exists
train_pairs -> ✅ exists
test_pairs -> ✅ exists


In [18]:
import os, numpy as np, pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
import faiss
from transformers import EsmTokenizer, EsmForMaskedLM, AutoTokenizer, AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# ========= 1) 候选库（全量） =========
all_prot_df = pairs[["target__uniprot_id","protein_seq"]].drop_duplicates("target__uniprot_id").reset_index(drop=True)
all_mol_df  = pairs[["compound__smiles"]].drop_duplicates("compound__smiles").reset_index(drop=True)

prot_ids = all_prot_df["target__uniprot_id"].tolist()
prot_seq = dict(zip(all_prot_df["target__uniprot_id"], all_prot_df["protein_seq"]))
smiles_list = all_mol_df["compound__smiles"].tolist()

prot2i = {p:i for i,p in enumerate(prot_ids)}
mol2i  = {s:i for i,s in enumerate(smiles_list)}

print("Total proteins:", len(prot_ids), "Total molecules:", len(smiles_list))

# ========= 2) 加载编码器 =========
PROT_MODEL = "westlake-repl/SaProt_650M_AF2"
MOL_MODEL  = "DeepChem/ChemBERTa-77M-MLM"

prot_tok = EsmTokenizer.from_pretrained(PROT_MODEL)
prot_mlm = EsmForMaskedLM.from_pretrained(PROT_MODEL).to(device).eval()

mol_tok = AutoTokenizer.from_pretrained(MOL_MODEL)
mol_enc = AutoModel.from_pretrained(MOL_MODEL).to(device).eval()

def aa_to_saprot_seq(aa: str) -> str:
    aa = aa.strip().upper().replace(" ", "")
    return "".join([c + "#" for c in aa if c.isalpha()])  # AA-only 最简输入

@torch.no_grad()
def protein_base_vec(aa_seq: str) -> np.ndarray:
    sap = aa_to_saprot_seq(aa_seq)
    x = prot_tok(sap, return_tensors="pt", truncation=True, max_length=1024)
    x = {k:v.to(device) for k,v in x.items()}
    out = prot_mlm(**x, output_hidden_states=True, return_dict=True)
    last = out.hidden_states[-1]
    mask = x["attention_mask"].unsqueeze(-1)
    pooled = (last * mask).sum(1) / mask.sum(1)
    return pooled[0].float().cpu().numpy()

@torch.no_grad()
def mol_base_vec(smiles: str) -> np.ndarray:
    x = mol_tok(smiles, return_tensors="pt", truncation=True, max_length=256)
    x = {k:v.to(device) for k,v in x.items()}
    out = mol_enc(**x, return_dict=True)
    last = out.last_hidden_state
    mask = x["attention_mask"].unsqueeze(-1)
    pooled = (last * mask).sum(1) / mask.sum(1)
    return pooled[0].float().cpu().numpy()

# ========= 3) 预计算 + 缓存 =========
os.makedirs("cache", exist_ok=True)
prot_cache = "cache/prot_base.npy"
mol_cache  = "cache/mol_base.npy"

if os.path.exists(prot_cache):
    prot_base = np.load(prot_cache)
    print("Loaded prot_base:", prot_base.shape)
else:
    prot_base = np.stack([protein_base_vec(prot_seq[p]) for p in tqdm(prot_ids, desc="Embed proteins")]).astype("float32")
    np.save(prot_cache, prot_base)
    print("Saved prot_base:", prot_base.shape)

if os.path.exists(mol_cache):
    mol_base = np.load(mol_cache)
    print("Loaded mol_base:", mol_base.shape)
else:
    mol_base = np.stack([mol_base_vec(s) for s in tqdm(smiles_list, desc="Embed molecules")]).astype("float32")
    np.save(mol_cache, mol_base)
    print("Saved mol_base:", mol_base.shape)

Hp, Hm = prot_base.shape[1], mol_base.shape[1]
print("Hp, Hm =", Hp, Hm)


device: cpu
Total proteins: 171 Total molecules: 558


Some weights of EsmForMaskedLM were not initialized from the model checkpoint at westlake-repl/SaProt_650M_AF2 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Embed proteins: 100%|██████████| 171/171 [41:13<00:00, 14.47s/it]


Saved prot_base: (171, 1280)


Embed molecules: 100%|██████████| 558/558 [00:06<00:00, 85.97it/s]

Saved mol_base: (558, 384)
Hp, Hm = 1280 384


In [19]:
import torch.nn as nn
import numpy as np
import torch

# 训练对：只用 train_pairs
train_pairs2 = train_pairs[train_pairs["target__uniprot_id"].isin(prot2i) & train_pairs["compound__smiles"].isin(mol2i)]
pair_p = np.array([prot2i[p] for p in train_pairs2["target__uniprot_id"]], dtype=np.int64)
pair_m = np.array([mol2i[s]  for s in train_pairs2["compound__smiles"]], dtype=np.int64)
print("train used pairs:", len(pair_p))

prot_base_t = torch.from_numpy(prot_base).to(device)
mol_base_t  = torch.from_numpy(mol_base).to(device)

d = 256
proj_p = nn.Linear(Hp, d, bias=False).to(device)
proj_m = nn.Linear(Hm, d, bias=False).to(device)

opt = torch.optim.AdamW(list(proj_p.parameters()) + list(proj_m.parameters()), lr=1e-3, weight_decay=1e-4)
temp = 0.07

# CPU / GPU 自动调参
if device == "cpu":
    STEPS, BATCH = 800, 128
else:
    STEPS, BATCH = 3000, 512

proj_p.train(); proj_m.train()
for step in range(STEPS):
    idx = np.random.randint(0, len(pair_p), size=min(BATCH, len(pair_p)))
    bp = torch.from_numpy(pair_p[idx]).to(device)
    bm = torch.from_numpy(pair_m[idx]).to(device)

    p = F.normalize(proj_p(prot_base_t[bp]), dim=-1)
    m = F.normalize(proj_m(mol_base_t[bm]),  dim=-1)

    logits = (p @ m.T) / temp
    labels = torch.arange(logits.size(0), device=device)
    loss = 0.5*(F.cross_entropy(logits, labels) + F.cross_entropy(logits.T, labels))

    opt.zero_grad(); loss.backward(); opt.step()

    if (step+1) % (STEPS//5) == 0:
        print(f"step {step+1}/{STEPS} loss={loss.item():.4f}")

proj_p.eval(); proj_m.eval()
with torch.no_grad():
    P_emb = F.normalize(proj_p(prot_base_t), dim=-1).float().cpu().numpy().astype("float32")
    M_emb = F.normalize(proj_m(mol_base_t),  dim=-1).float().cpu().numpy().astype("float32")

mol_index  = faiss.IndexFlatIP(d); mol_index.add(M_emb)
prot_index = faiss.IndexFlatIP(d); prot_index.add(P_emb)

def search_mols_by_protein(uniprot_id, topk=10):
    i = prot2i[uniprot_id]
    D, I = mol_index.search(P_emb[i:i+1], int(topk))
    return [smiles_list[int(j)] for j in I[0]]

def search_prots_by_smiles(smiles, topk=10):
    i = mol2i[smiles]
    D, I = prot_index.search(M_emb[i:i+1], int(topk))
    return [prot_ids[int(j)] for j in I[0]]

print("✅ trained projections + built indices")


train used pairs: 3686
step 160/800 loss=4.1234
step 320/800 loss=3.9440
step 480/800 loss=4.0375
step 640/800 loss=3.6789
step 800/800 loss=3.7508
✅ trained projections + built indices


In [20]:
from tqdm import tqdm

# test：按蛋白聚合正例分子集合
test_pos_mols = test_pairs.groupby("target__uniprot_id")["compound__smiles"].apply(set).to_dict()

def recall_p2m(k=10):
    accs = list(test_pos_mols.keys())
    hit = 0
    for acc in tqdm(accs, desc=f"Test P->M Recall@{k}"):
        rec = search_mols_by_protein(acc, topk=k)
        if len(test_pos_mols[acc].intersection(rec)) > 0:
            hit += 1
    return hit / max(1, len(accs))

print("Test P->M Recall@1 :", recall_p2m(1))
print("Test P->M Recall@5 :", recall_p2m(5))
print("Test P->M Recall@10:", recall_p2m(10))


Test P->M Recall@1: 100%|██████████| 35/35 [00:00<00:00, 9461.85it/s]


Test P->M Recall@1 : 0.14285714285714285


Test P->M Recall@5: 100%|██████████| 35/35 [00:00<00:00, 5108.60it/s]


Test P->M Recall@5 : 0.6


Test P->M Recall@10: 100%|██████████| 35/35 [00:00<00:00, 7187.30it/s]

Test P->M Recall@10: 0.8285714285714286


In [21]:
!pip -q install gradio


In [23]:
import gradio as gr
import pandas as pd

def ui_p2m(uniprot_id, k):
    res = search_mols_by_protein(uniprot_id, topk=int(k))
    if len(res) > 0 and isinstance(res[0], tuple):
        return pd.DataFrame({"rank": range(1,len(res)+1),
                             "score": [x[0] for x in res],
                             "smiles": [x[1] for x in res]})
    else:
        return pd.DataFrame({"rank": range(1,len(res)+1), "smiles": res})

def ui_m2p(smiles, k):
    res = search_prots_by_smiles(smiles, topk=int(k))
    if len(res) > 0 and isinstance(res[0], tuple):
        return pd.DataFrame({"rank": range(1,len(res)+1),
                             "score": [x[0] for x in res],
                             "uniprot_id": [x[1] for x in res]})
    else:
        return pd.DataFrame({"rank": range(1,len(res)+1), "uniprot_id": res})

with gr.Blocks() as demo:
    gr.Markdown("## Task1 Demo: Bidirectional Retrieval (Protein ↔ Molecule)")
    gr.Markdown("Select a Protein (UniProt ID) or a Molecule (SMILES) and retrieve Top-K matches.")

    with gr.Tab("Protein → Molecule"):
        p = gr.Dropdown(choices=prot_ids, label="Protein (UniProt ID)")
        k1 = gr.Slider(1, 50, value=10, step=1, label="Top-K")
        out1 = gr.Dataframe(label="Retrieved Molecules", interactive=False)
        gr.Button("Search").click(ui_p2m, [p, k1], out1)

    with gr.Tab("Molecule → Protein"):
        s = gr.Dropdown(choices=smiles_list, label="Molecule (SMILES)")
        k2 = gr.Slider(1, 50, value=10, step=1, label="Top-K")
        out2 = gr.Dataframe(label="Retrieved Proteins", interactive=False)
        gr.Button("Search").click(ui_m2p, [s, k2], out2)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2144ca3e73d81e1158.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
